In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import xarray as xr
import pingouin as pg
import re
from tqdm import tqdm
from os.path import join as pjoin
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import pearsonr, spearmanr, zscore, wilcoxon
import sys
import os

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
sys.path.append('..')
import circletrack_neural as ctn
import circletrack_behavior as ctb
import pca_ica as ica
import plotting_functions as pf

In [ ]:
path = '../../EnsembleRemodeling_Resubmission/circletrack_data'
mouse_list = ['mc03', 'mc06', 'mc07', 'mc09', 'mc11']   
session_dict = {'mc03': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal4', 'Training_Reversal'],
                'mc06': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal'],
                'mc07': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal'],
                'mc09': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal'],
                'mc11': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal']}
cohort_number = 'cohort1'
session_length = '30min'
# ## Loop through all mice
# for mouse in mouse_list:
#     dpath = pjoin(path, 'Results/{}/'.format(mouse))
#     session_dates = os.listdir(dpath)
#     session_dates.sort()
#     for i, date in enumerate(session_dates):
#         ctn.minian_to_netcdf(path, mouse, date, session_id = session_dict[mouse][i], cohort_number = cohort_number, session_length = session_length,
#                              sampling_rate = 1/15, down_sample_factor = 2)

In [48]:
## Create a list of sessions for easy plotting later and for changing column order
session_list = ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal']
## Create session_id dictionary
session_dict = {'mc03': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal4', 'Training_Reversal'],
                'mc06': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal'],
                'mc07': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal'],
                'mc09': ['Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal'],
                'mc11': ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1', 'Reversal2', 'Reversal3', 'Reversal4', 'Training_Reversal']}
## Set path variables
ensemble_path = '../../EnsembleRemodeling_Resubmission/circletrack_data/assemblies'
behavior_path = '../../EnsembleRemodeling_Resubmission/circletrack_data/'  
figure_path = '../../EnsembleRemodeling_Resubmission/circletrack_data/python_analysis/progress_figures/cohort1'
## Create mouse list
mouse_list = ['mc03', 'mc06', 'mc07', 'mc09', 'mc11']             

In [ ]:
## Number of neurons across sessions
## Set mouse list
mouse_list = ['mc03', 'mc06', 'mc07']
mouse_neural = {}
## Loop through mouse list
for mouse in mouse_list:
    sessions = ctn.import_mouse_neural_data(behavior_path, mouse, key_file = 'keys.yml', session = '30min', neural_type = 'spikes')
    cell_dict = {}
    for session in sessions:
        cell_dict[session] = len(sessions[session].unit_id)
    mouse_neural[mouse] = cell_dict
fig = pf.custom_graph_template(title = 'Cells per Session', x_title = 'Session', y_title = 'Number of Cells')
for mouse in mouse_neural:
    plot_data = pd.DataFrame(mouse_neural[mouse], index = [0])
    if mouse == 'mc03':
        plot_data.insert(0, 'Reversal2', np.nan)
        plot_data.insert(1, 'Reversal3', np.nan)
    plot_data = plot_data[session_list]
    fig.add_trace(go.Scatter(x = session_list, y = plot_data.iloc[0, :], mode = 'lines', line_color = 'grey', line_width = 0.5, opacity = 0.5, name = mouse, showlegend = False))
fig.show()
fig.write_image('progress_figures/cells_across_session.png')

In [ ]:
## Plot number of ensembles across all sessions
ensemble_num_df = pd.DataFrame()
mouse_name = []
for i, mouse in enumerate(mouse_num_ensembles):
    mouse_name.append(mouse)
    df = pd.DataFrame(mouse_num_ensembles[mouse], index = [i])
    ensemble_num_df = pd.concat([ensemble_num_df, df])
ensemble_num_df = ensemble_num_df[session_list]
## Calculate avg num of ensembles per session
avg = ensemble_num_df.mean()
## Plot data
fig = pf.custom_graph_template(title = 'Ensembles Across Sessions', x_title = '', y_title = 'Number of Ensembles', width = 500, height = 500)
fig.add_trace(go.Scatter(x = session_list, y = avg, marker = dict(color = 'turquoise'), showlegend = False))
## Plot individual mice
for mouse in mouse_num_ensembles:
    fig.add_trace(go.Scatter(x = session_list, y = pd.Series(mouse_num_ensembles[mouse]), mode = 'lines', line_color = 'grey', opacity = 0.5, line_width = 0.5, showlegend = False, name = mouse))
fig.show()
fig.write_image('progress_figures/ensembles_across_sessions.png')

In [ ]:
## Set mouse list, x_bin_size, and across_time
mouse_list = ['mc03', 'mc06', 'mc07', 'mc09']
across_time = True
x_bin_size = 5
## Create empty dictionaries
mouse_trends = {}
mouse_binned_activations = {}
mouse_slopes = {}
mouse_taus = {}
mouse_ensembles = {}
mouse_num_ensembles = {}
## Loop through each mouse
for mouse in tqdm(mouse_list):
    ## saved ensemble data path
    spath = pjoin(ensemble_path, mouse)
    ## load in behavior data across all sessions
    if not across_time:
        mouse_behavior = ctb.import_mouse_behavior_data(behavior_path, mouse, key_file = 'behavior_keys.yml', session = '30min')
    ## Create empty dictionaries to store output
    determined_trends = {}
    binned_activations_dict = {}
    slopes_dict = {}
    tau_dict = {}
    num_ensembles = {}
    ## Load assemblies
    for session in session_dict[mouse]:
        ## Load a specific session's behavior data
        if not across_time:
            aligned_behavior = mouse_behavior[session]
            ## Get which timestamps are part of which trial for the aligned behavior data
            trials = ctb.get_trials(aligned_behavior, shift_factor = (np.pi / 2), angle_type = 'radians', counterclockwise = True)
        ## Load a specific session's assemblies
        assemblies = ica.load_session_assemblies(mouse, spath = spath, session_id = session)
        act = assemblies.activations.values
        ## Save the number of identified ensembles by PCA/ICA
        num_ensembles[session] = act.shape[0]
        if across_time:
          trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_time(act, z_threshold = None, x_bin_size = x_bin_size, analysis_type = 'max', zscored = True, alpha = 'sidak')  
        else:
            ## Define ensemble trends across trials to determine if activation strength is increasing/decreasing across the session
            trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_trials(act, aligned_behavior, trials, trial_type = 'forward', z_threshold = None)
        ## Save to dictionaries
        determined_trends[session] = trends
        binned_activations_dict[session] = binned_activations
        slopes_dict[session] = slopes
        tau_dict[session] = tau
    ## Determine the proportion of ensembles that are increasing, decreasing, or have no trend based on their activation strength across time
    proportion_dict = ica.calculate_proportions_ensembles(determined_trends)
    ## Save to mouse dictionaries before looping to the next mouse
    mouse_trends[mouse] = proportion_dict
    mouse_binned_activations[mouse] = binned_activations_dict
    mouse_slopes[mouse] = slopes_dict
    mouse_ensembles[mouse] = determined_trends
    mouse_taus[mouse] = tau_dict
    mouse_num_ensembles[mouse] = num_ensembles

In [ ]:
## Plot peak activation of a fading ensemble
activations = mouse_binned_activations['mc09']['Reversal1']
ensemble_number = 5
fig = pf.plot_activation_strength(activations, ensemble_number = ensemble_number, figure_path = figure_path, x_bin_size = x_bin_size,
                                  title = 'Fading Ensemble', y_title = 'Max Z Score', x_title = 'Time (s)', 
                                  file_name = 'max_strength_fading_{}bins.png'.format(x_bin_size), marker_color = 'red')

In [ ]:
## Plot peak activation of an increasing ensemble
activations = mouse_binned_activations['mc09']['Reversal1']
ensemble_number = 28
fig = pf.plot_activation_strength(activations, ensemble_number = ensemble_number, figure_path = figure_path, x_bin_size = x_bin_size,
                                  title = 'Increasing Ensemble', y_title = 'Max Z Score', x_title = 'Time (s)', 
                                  file_name = 'max_strength_increasing_{}bins.png'.format(x_bin_size), marker_color = 'turquoise')

In [ ]:
## Plot peak activation of a no-trend ensemble
activations = mouse_binned_activations['mc09']['Reversal1']
ensemble_number = 1
fig = pf.plot_activation_strength(activations, ensemble_number = ensemble_number, figure_path = figure_path, x_bin_size = x_bin_size,
                                  title = 'No Trend Ensemble', y_title = 'Max Z Score', x_title = 'Time (s)', 
                                  file_name = 'max_strength_notrend_{}bins.png'.format(x_bin_size), marker_color = 'darkgrey')

In [ ]:
## Get proportion decreasing values
prop_decreasing_T4 = []
prop_decreasing_R1 = []
for key in mouse_trends:
    prop_decreasing_T4.append(mouse_trends[key]['Training4']['prop_decreasing'])
    prop_decreasing_R1.append(mouse_trends[key]['Reversal1']['prop_decreasing'])
prop_df = pd.DataFrame({'Training4': prop_decreasing_T4, 'Reversal1': prop_decreasing_R1})
## Plot prop_df
x = ['Training4', 'Reversal1']
avgs = [prop_df.Training4.mean(), prop_df.Reversal1.mean()]
sem = [prop_df.Training4.sem(), prop_df.Reversal1.sem()]
fig = go.Figure(data = go.Bar(x = x, y = avgs, error_y = dict(type = 'data', array = sem), showlegend = False))
for row in prop_df.index:
    data = prop_df.loc[row]
    fig.add_trace(go.Scatter(x = x, y = [data.Training4, data.Reversal1], mode = 'lines + markers', showlegend = False,
                             line = dict(width = 1)))
fig.update_layout(template = 'simple_white', width = 500, height = 500)
fig.update_layout(
    title={
        'text': 'Fading Ensembles',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_yaxes(title = 'Proportion Fading Ensembles')
fig.update_traces(marker_color = 'red', marker_line_color = 'black', marker_line_width = 2)
fig.show()
fig.write_image('progress_figures/binned_by_15seconds.png')

In [ ]:
## Get proportion fading ensembles for last day, since some mice went back to Training reward port contingencies
training_group = []
reversal_group = []
mice_training = ['mc06', 'mc09', 'mc11']
mice_reversal = ['mc03', 'mc07']
for mouse in mice_training:
    training_group.append(mouse_trends[mouse]['Training_Reversal']['prop_decreasing'])
for mouse in mice_reversal:
    reversal_group.append(mouse_trends[mouse]['Training_Reversal']['prop_decreasing'])
## Add NaN to make mice_reversal same length as mice_training
reversal_group.append(np.nan)
## Create dataframe
df = pd.DataFrame({'Training_Contingency': training_group, 'Reversal_Contingency': reversal_group})
avgs = [df.Training_Contingency.mean(), df.Reversal_Contingency.mean()]
sem = [df.Training_Contingency.sem(), df.Reversal_Contingency.sem()]
## Plot figure
x = ['Training Contingency', 'Reversal Contingency']
fig = go.Figure(data = go.Bar(x = x, y = avgs, error_y = dict(type = 'data', array = sem)))
fig.update_layout(template = 'simple_white', width = 500, height = 500)
fig.update_layout(
    title={
        'text': 'Fading Ensembles',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_yaxes(title = 'Proportion Fading Ensembles')
fig.update_traces(marker_color = 'red', marker_line_color = 'black', marker_line_width = 2)
fig.show()
fig.write_image('progress_figures/training_contingency_15seconds.png')

In [ ]:
## Look at average number of fading ensembles across all training sessions and across all reversal sessions
trainings = ['Training1', 'Training2', 'Training3', 'Training4']
reversals = ['Reversal1', 'Reversal2', 'Reversal3', 'Reversal4']
prop_dec_training = []
prop_dec_reversal = []
mouse_list = []
for mouse in mouse_trends:
    for session in trainings:
        prop_dec_training.append(mouse_trends[mouse][session]['prop_decreasing'])
        mouse_list.append(mouse)
    ## Because mc03 doesn't have Reversal2 or Reversal3 due to miniscope issues, add only Reversal1 and Reversal4 manually
    if mouse == 'mc03':
            prop_dec_reversal.append(mouse_trends[mouse][reversals[0]]['prop_decreasing'])
            prop_dec_reversal.append(mouse_trends[mouse][reversals[3]]['prop_decreasing'])
            ## Add NaN to prop_dec_reversal to make it same length as prop_dec_training
            prop_dec_reversal.append(np.nan)
            prop_dec_reversal.append(np.nan)
    else:
        for reverse in reversals:
            prop_dec_reversal.append(mouse_trends[mouse][reverse]['prop_decreasing'])
## Create dataframe
df = pd.DataFrame({'Mouse': mouse_list, 'Trainings': prop_dec_training, 'Reversals': prop_dec_reversal})
## melt df for plotting
df_melt = pd.melt(df, id_vars = ['Mouse'], value_vars = ['Trainings', 'Reversals'], var_name = 'session')
## Averages by mouse
avgs = df.groupby(['Mouse'], as_index = False).mean(numeric_only = True)
sem = df.groupby(['Mouse'], as_index = False).sem()
## Plot groups
pf.plot_across_groups(df_melt, groupby = 'Mouse', separateby = 'session', plot_var = 'value', colors = 'red', title = 'Fading Ensembles Across Each Epoch', y_title = 'Proportion Fading Ensembles',
                      save_path = '/media/caishuman/csstorage3/Austin/CircleTrack/EnsembleRemodeling_Resubmission/CircleTrack_Data/python_analysis/progress_figures/fading_across_all_15seconds.png')

In [ ]:
## Plot three ensembles with their members colored 
fig = pf.stem_plot(assemblies['patterns'][1], member_color = 'turquoise', size = 3)
fig1 = pf.stem_plot(assemblies['patterns'][4], member_color = 'turquoise', size = 3)
fig2 = pf.stem_plot(assemblies['patterns'][7], member_color = 'turquoise', size = 3)
fig.update_layout(height = 300, width = 500)
fig1.update_layout(height = 300, width = 500)
fig2.update_layout(height = 300, width = 500)
fig.update_layout(
    title={
        'text': 'Ensemble Membership',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_yaxes(range = [-0.55, 0.55], tickvals=[0.4, 0.2, 0, -0.2, -0.4])
fig1.update_yaxes(range = [-0.55, 0.55], tickvals=[0.4, 0.2, 0, -0.2, -0.4])
fig2.update_yaxes(range = [-0.55, 0.55], tickvals=[0.4, 0.2, 0, -0.2, -0.4])
fig.show()
fig.write_image('progress_figures/ensemble_membership_top.png')
fig1.show()
fig1.write_image('progress_figures/ensemble_membership_one.png')
fig2.show()
fig2.write_image('progress_figures/ensemble_membership_two.png')

In [ ]:
## Plot fading ensembles across all sessions
## Create empty dictionary
trends_dict = {}
## Loop through mouse_trends and extract the proportion of fading ensembles for each session (Training1, Training2, etc)
for key in mouse_trends:
    fading_ensembles = []
    for session in mouse_trends[key]:
        fading_ensembles.append(mouse_trends[key][session]['prop_decreasing'])
    trends_dict[key] = fading_ensembles
## Add in NaN values for days that are missing for mc03
trends_dict['mc03'][5] = np.nan
trends_dict['mc03'][6] = np.nan
trends_dict['mc03'].append(0.0) ## value for proportion of fading ensembles on Reversal4
trends_dict['mc03'].append(0.029411764705882353) ## value for proportion of fading ensembles on Training_Reversal
## Plot values for each mouse
fig = go.Figure()
x_axis = session_list
for key in trends_dict:
    fig.add_trace(go.Scatter(x = x_axis, y = trends_dict[key], name = key))
fig.update_layout(template = 'simple_white', height = 500, width = 500)
fig.update_layout(
    title={
        'text': 'Fading Ensemble Proportion Across Sessions',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_yaxes(title = 'Proportion Fading Ensembles')
fig.show()
fig.write_image('progress_figures/fading_all_sessions_15seconds.png')

The below code is to subset the neural data across FORWARD TRIALS and detect fading ensembles using trials to bin.

In [42]:
## Set mouse list, path, and z_thresh for binarization
mouse_list = ['mc03', 'mc06', 'mc07', 'mc09', 'mc11'] 
across_time = False
analysis_type = 'max'
alpha_old = 0.05
## Create empty dictionaries
mouse_trends = {}
mouse_binned_activations = {}
mouse_slopes = {}
mouse_taus = {}
mouse_ensembles = {}
mouse_trial_times = {}
## Loop through each mouse
for mouse in tqdm(mouse_list):
    ## Create empty dictionaries to store output
    determined_trends = {}
    binned_activations_dict = {}
    slopes_dict = {}
    tau_dict = {}
    trial_times = {}
    ensemble_path = '../../EnsembleRemodeling_Resubmission/circletrack_data/assemblies'
    behav_path = '../../EnsembleRemodeling_Resubmission/circletrack_data/output/behav'  
    behav_path = pjoin(behav_path, '{}'.format(mouse))
    ensemble_path = pjoin(ensemble_path, '{}'.format(mouse))
    ## Load assemblies
    for session in session_dict[mouse]:
        ## Load specific session's assemblies
        assemblies = ica.load_session_assemblies(mouse, ensemble_path, session)
        ## Set activation values as act
        act = assemblies.activations.values
        ## Load a specific session's behavior data
        if not across_time:
            behav_file = pjoin(behav_path, '{}_{}.feat'.format(mouse, session))
            behavior_df = pd.read_feather(behav_file)
            aligned_behavior = ctb.align_behavior_frames(behavior_df, session='30min', sampling_rate=1/15)
            ## Ensure that activations and aligned behavior are the same length
            aligned_behavior = ica.align_activations_to_behavior(act, aligned_behavior)
            ## Get which timestamps are part of which trial for the aligned behavior data
            trials = aligned_behavior.trials
            ## Get length of time for each trial
            time_diff = []
            for trial in np.unique(trials):
                ## Subset aligned_behavior by a given trial
                behavior = aligned_behavior.loc[trials == trial]
                ## Get the first and last timestamp to determine the window
                first_timestamp, last_timestamp = behavior.t.to_numpy()[0], behavior.t.to_numpy()[-1]
                ## Convert from ms to s
                first_timestamp = first_timestamp
                last_timestamp = last_timestamp
                ## Append to time_diff list
                time_diff.append(last_timestamp - first_timestamp)
            trial_times[session] = time_diff
        ## This is where the data gets binned either by even time intervals or by trials
        if across_time:
          trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_time(act, z_threshold = None, x_bin_size = x_bin_size, analysis_type = analysis_type, 
                                                                                           zscored = True, alpha_old = alpha_old)  
        else:
            ## Define ensemble trends across trials to determine if activation strength is increasing/decreasing across the session
            trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_trials(act, aligned_behavior, trials, trial_type = 'all', 
                                                                                               z_threshold = None, alpha_old = alpha_old, analysis_type = analysis_type)
        ## Save to dictionaries
        determined_trends[session] = trends
        binned_activations_dict[session] = binned_activations
        slopes_dict[session] = slopes
        tau_dict[session] = tau
    ## Determine the proportion of ensembles that are increasing, decreasing, or have no trend based on their activation strength across time
    proportion_dict = ica.calculate_proportions_ensembles(determined_trends)
    ## Save to mouse dictionaries before looping to the next mouse
    mouse_trends[mouse] = proportion_dict
    mouse_binned_activations[mouse] = binned_activations_dict
    mouse_slopes[mouse] = slopes_dict
    mouse_taus[mouse] = tau_dict
    mouse_ensembles[mouse] = determined_trends
    mouse_trial_times[mouse] = trial_times

  0%|          | 0/5 [00:00<?, ?it/s]/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/statsmodels/stats/multitest.py:155: RuntimeWarning: divide by zero encountered in log1p
  pvals_corrected = -np.expm1(ntests * np.log1p(-pvals))
/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/statsmodels/stats/multitest.py:155: RuntimeWarning: divide by zero encountered in log1p
  pvals_corrected = -np.expm1(ntests * np.log1p(-pvals))
 20%|██        | 1/5 [00:41<02:45, 41.44s/it]


KeyError: '[63014, 63016, 63019, 63021, 63023, 63025, 63027, 63029, 63031, 63034, 63036, 63038, 63040, 63042, 63045, 63047, 63049, 63051, 63053, 63055, 63058, 63060, 63062, 63064, 63067, 63069, 63071, 63074, 63076, 63078, 63080, 63082, 63085, 63087, 63089, 63091, 63094, 63096, 63098, 63100, 63103, 63105, 63107, 63110, 63112, 63114, 63116, 63119, 63121, 63123, 63125, 63127, 63129, 63132, 63134, 63136, 63138, 63140, 63142, 63145, 63147, 63149, 63151, 63153, 63156, 63158, 63160, 63162, 63164, 63167, 63169, 63171, 63173, 63175, 63178, 63180, 63182, 63185, 63188, 63190, 63192, 63195, 63197, 63199, 63201, 63203, 63206, 63208, 63210, 63213, 63215, 63217, 63219, 63222, 63224, 63226, 63228, 63230, 63233, 63235, 63237, 63239, 63241, 63243, 63246, 63248, 63250, 63252, 63257, 63259, 63262, 63266, 63268, 63270, 63272, 63275, 63277, 63279, 63281, 63284, 63286, 63288, 63290, 63293, 63295, 63297, 63300, 63302, 63304, 63306, 63308, 63311, 63313, 63315, 63317, 63319, 63321, 63324, 63326, 63328, 63330, 6

In [44]:
behavior_df
arg_mins = [np.abs(behavior_df['t'] - (t)).argmin() for t in time]
lined_up_timeframes = np.array(behavior_df.loc[arg_mins, 'frame']) 
lined_up_timeframes

array([    1,     3,     5, ..., 64678, 64682, 64684])

In [61]:
rewards = behavior_df.loc[behavior_df['water'] == True]
reward_one, reward_two = np.unique(rewards['lick_port'])[0], np.unique(rewards['lick_port'])[1]

In [64]:
reward_two

5

In [ ]:
## Plot peak activation of a fading ensemble
activations = mouse_binned_activations['mc09']['Reversal1']
ensemble_number = 1
fig = pf.plot_activation_strength(activations, ensemble_number = ensemble_number, figure_path = figure_path, x_bin_size = None,
                                  title = 'Fading Ensemble', y_title = 'Max Z Score', x_title = 'Trials', 
                                  file_name = 'max_strength_fading_forward_trials.png'.format(x_bin_size), marker_color = 'red')

In [ ]:
## Plot peak activation of an increasing ensemble
activations = mouse_binned_activations['mc09']['Reversal1']
ensemble_number = 57
fig = pf.plot_activation_strength(activations, ensemble_number = ensemble_number, figure_path = figure_path, x_bin_size = None,
                                  title = 'Increasing Ensemble', y_title = 'Max Z Score', x_title = 'Trials', 
                                  file_name = 'max_strength_increasing_forward_trials.png'.format(x_bin_size), marker_color = 'turquoise')

In [ ]:
## Plot peak activation of a no-trend ensemble
activations = mouse_binned_activations['mc09']['Reversal1']
ensemble_number = 28
fig = pf.plot_activation_strength(activations, ensemble_number = ensemble_number, figure_path = figure_path, x_bin_size = None,
                                  title = 'No Trend Ensemble', y_title = 'Max Z Score', x_title = 'Trials', 
                                  file_name = 'max_strength_notrend_forward_trials.png'.format(x_bin_size), marker_color = 'darkgrey')

In [ ]:
## Get proportion decreasing values
prop_decreasing_T4 = []
prop_decreasing_R1 = []
mouse_name = []
for key in mouse_trends:
    prop_decreasing_T4.append(mouse_trends[key]['Training4']['prop_decreasing'])
    prop_decreasing_R1.append(mouse_trends[key]['Reversal1']['prop_decreasing'])
    mouse_name.append(key)
prop_df = pd.DataFrame({'Mouse': mouse_name, 'Training4': prop_decreasing_T4, 'Reversal1': prop_decreasing_R1})
## Plot prop_df
x = ['Training4', 'Reversal1']
avgs = [prop_df.Training4.mean(), prop_df.Reversal1.mean()]
sem = [prop_df.Training4.sem(), prop_df.Reversal1.sem()]
fig = go.Figure(data = go.Bar(x = x, y = avgs, error_y = dict(type = 'data', array = sem), showlegend = False))
for row in prop_df.index:
    data = prop_df.loc[row]
    fig.add_trace(go.Scatter(x = x, y = [data.Training4, data.Reversal1], mode = 'lines', line_color = 'grey', showlegend = False,
                             line = dict(width = 1), name = prop_df.loc[row, 'Mouse']))
fig.update_layout(template = 'simple_white', width = 500, height = 500)
fig.update_layout(
    title={
        'text': 'Fading Ensembles by Forward Trials',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_yaxes(title = 'Proportion Fading Ensembles')
fig.update_traces(marker_color = 'red', marker_line_color = 'black', marker_line_width = 2)
fig.show()
# fig.write_image(pjoin(figure_path, 'proportion_fading_trials_max.png'))

In [ ]:
wilcoxon_result = wilcoxon(x = prop_df.Training4, y = prop_df.Reversal1, method = 'approx')
wilcoxon_result

In [ ]:
## Get proportion fading ensembles for last day, since some mice went back to Training reward port contingencies
training_group = []
reversal_group = []
mice_training = ['mc06', 'mc09']
mice_reversal = ['mc03', 'mc07']
for mouse in mice_training:
    training_group.append(mouse_trends[mouse]['Training_Reversal']['prop_decreasing'])
for mouse in mice_reversal:
    reversal_group.append(mouse_trends[mouse]['Training_Reversal']['prop_decreasing'])
## Create dataframe
df = pd.DataFrame({'Training_Contingency': training_group, 'Reversal_Contingency': reversal_group})
avgs = [df.Training_Contingency.mean(), df.Reversal_Contingency.mean()]
sem = [df.Training_Contingency.sem(), df.Reversal_Contingency.sem()]
## Plot figure
x = ['Training Contingency', 'Reversal Contingency']
fig = go.Figure(data = go.Bar(x = x, y = avgs, error_y = dict(type = 'data', array = sem)))
fig.update_layout(template = 'simple_white', width = 500, height = 500)
fig.update_layout(
    title={
        'text': 'Fading Ensembles by Forward Trials',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_yaxes(title = 'Proportion Fading Ensembles')
fig.update_traces(marker_color = 'red', marker_line_color = 'black', marker_line_width = 2)
fig.show()
fig.write_image(pjoin(figure_path, 'proportion_fading_last_day_trials.png'))

In [ ]:
## Plot fading ensembles across all sessions
## Create empty dictionary
trends_dict = {}
## Loop through mouse_trends and extract the proportion of fading ensembles for each session (Training1, Training2, etc)
for key in mouse_trends:
    fading_ensembles = []
    for session in mouse_trends[key]:
        fading_ensembles.append(mouse_trends[key][session]['prop_decreasing'])
    trends_dict[key] = fading_ensembles
## Add in NaN values for days that are missing for mc03
trends_dict['mc03'][5] = np.nan
trends_dict['mc03'][6] = np.nan
trends_dict['mc03'].append(0.0) ## value for proportion of fading ensembles on Reversal4
trends_dict['mc03'].append(0.0) ## value for proportion of fading ensembles on Training_Reversal
## Plot values for each mouse
fig = go.Figure()
x_axis = session_list
for key in trends_dict:
    fig.add_trace(go.Scatter(x = x_axis, y = trends_dict[key], name = key))
fig.update_layout(template = 'simple_white', height = 500, width = 500)
fig.update_layout(
    title={
        'text': 'Fading Ensemble Proportion Across Sessions',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_yaxes(title = 'Proportion Fading Ensembles')
fig.show()
fig.write_image(pjoin(figure_path, 'all_sessions_trials.png'))

Correlate correct rejection rate with proportion of fading ensembles.

In [ ]:
## Set relative path variable for circletrack behavior data
path = pjoin(behavior_path, 'Data/**/**/**/circle_track.csv')
## Set str2match variable (regex for mouse name)
str2match = '(mc[0-9]+)'
## Create list of files
file_list = ctb.get_file_list(path)
## Loop through file_list to extract mouse name
mouseID = []
for file in file_list:
    mouse = ctb.get_mouse(file, str2match)
    mouseID.append(mouse)
## Combine file_list and mouseID
combined_list = ctb.combine(file_list, mouseID)
## dprime across all mice and all sessions
mouse_list = ['mc03', 'mc06', 'mc07', 'mc09', 'mc11']
bin_size = 1
mouse_signal_data = {}
mouse_agg_data = {}
mouse_signal_forward = {}
mouse_agg_forward = {}
mouse_trial_times = {}
for mouse in mouse_list:
    files = ctb.subset_combined(combined_list, mouse)
    signal_dict_all = {}
    signal_dict_forward = {}
    agg_dict_all = {}
    agg_dict_forward = {}
    trial_length_session = {}
    for path in files:
        ## Extract date
        date = re.search('(2022_[0-9]+_[0-9]+)', path).group()
        ## Load circle track data
        circle_track = pd.read_csv(path)
        ## Get which ports are the reward ports
        reward_one, reward_two = ctb.get_rewarding_ports(circle_track)
        ## Normalize timestamps and crop data
        circle_track = ctb.normalize_timestamp(circle_track)
        circle_track = ctb.crop_data(circle_track)
        ## Align behavior data
        aligned_behavior = ctb.load_and_align_behavior(behavior_path, mouse, date, session = '30min', plot_frame_usage = False)
        ## Get timestamps where mouse licked (replaces REWARD with LICK)
        lick_tmp = ctb.get_licks(circle_track)
        ## Calculate trials, label each frame as a specific trial
        trials = ctb.get_trials(aligned_behavior, shift_factor = 0, angle_type = 'radians', counterclockwise = True)
        ## Calculate forward and reverse trials
        forward_trials, reverse_trials = ctb.forward_reverse_trials(aligned_behavior, trials, positive_jump = 350, wiggle = 3)
        ## Label each frame in aligned_behavior with a trial
        aligned_behavior.insert(1, 'trial', trials)
        ## Calculate trial times
        trial_length_dict = ctb.calculate_trial_times(aligned_behavior, forward_trials)
        ## Label lick data with what trial those licks occurred during
        lick_data = ctb.label_lick_trials(aligned_behavior, lick_tmp, trials)
        ## Calculate dprime, CR, FA, hits, misses for each trial
        signal_all = ctb.dprime_metrics(lick_data, trials, reward_one, reward_two)
        ## Calculate for only trials in the correct direction
        signal_forward = ctb.dprime_metrics(lick_data, forward_trials, reward_one, reward_two, reward_index = 'one')
        ## Aggregate data every 5 trials
        agg_data = ctb.aggregate_metrics(signal_all, bin_size = bin_size)
        ## Aggregate data over forward trials
        agg_data_forward = ctb.aggregate_metrics(signal_forward, bin_size = bin_size)
        ## Append to dictionaries
        signal_dict_all[date] = signal_all
        signal_dict_forward[date] = signal_forward
        agg_dict_all[date] = agg_data
        agg_dict_forward[date] = agg_data_forward
        trial_length_session[date] = trial_length_dict
    ## Append to mouse dictionaries
    mouse_signal_data[mouse] = signal_dict_all
    mouse_signal_forward[mouse] = signal_dict_forward
    mouse_agg_data[mouse] = agg_dict_all
    mouse_agg_forward[mouse] = mouse_agg_forward
    mouse_trial_times[mouse] = trial_length_session

In [ ]:
## Create a scatter plot of trial length vs dprime
mouse_df = pd.DataFrame()
for mouse in mouse_list:
    combined_df = pd.DataFrame()
    for date in mouse_signal_forward[mouse]:
        df_trials = pd.DataFrame(mouse_trial_times[mouse][date])
        df_signal = pd.DataFrame(mouse_signal_forward[mouse][date])
        combined_df = pd.concat([df_trials, df_signal], axis = 1)
        combined_df.insert(7, 'mouse', mouse)
        combined_df.insert(8, 'session', date)
        mouse_df = pd.concat([mouse_df, combined_df])

In [ ]:
mouse_df

In [ ]:
fig = pf.custom_graph_template('', x_title = "Trial d'", y_title = "Trial Length (s)")
fig.add_trace(go.Scatter(x = mouse_df['dprime'], y = mouse_df['trial_length'], mode = 'markers'))
fig.show()
fig.write_image(pjoin(figure_path, 'dprime_vs_triallength_cohort1.png'))

In [ ]:
## Plot Correct Rejection Rate values across all sessions
mouse_cr = {}
for mouse in mouse_signal_forward:
    cr_values = {}
    for date in mouse_signal_forward[mouse]:
        cr_values[date] = np.nanmean(mouse_signal_forward[mouse][date]['CR'])
    mouse_cr[mouse] = cr_values
## For mc03, the dates are in a strange order, so reorder the keys
desired_ordered_list = ['2022_09_29', '2022_09_30', '2022_10_01', '2022_10_02', '2022_10_03', '2022_10_04', '2022_10_06', '2022_10_07']
mouse_cr['mc03'] = {k: mouse_cr['mc03'][k] for k in desired_ordered_list}
## Turn into a dataframe
cr_df = pd.DataFrame()
for i, mouse in enumerate(mouse_cr):
    data = pd.DataFrame(mouse_cr[mouse], index = [i])
    cr_df = pd.concat([cr_df, data])
## Reorder columns
columns = ['2022_09_29', '2022_09_30', '2022_10_01', '2022_10_02', '2022_10_03', '2022_10_04', '2022_10_05', '2022_10_06', '2022_10_07']
cr_df = cr_df[columns]
## Rename columns
cr_df = cr_df.rename(columns = {'2022_09_29': 'Training1', '2022_09_30': 'Training2', '2022_10_01': 'Training3', '2022_10_02': 'Training4', 
                                '2022_10_03': 'Reversal1', '2022_10_04': 'Reversal2', '2022_10_05': 'Reversal3', '2022_10_06': 'Reversal4', '2022_10_07': 'Training_Reversal'})
## Insert mice into df
cr_df.insert(0, 'mouse', mouse_list)
## Rename columns proportion fading ensembles df to insert into cr_df
prop_df_fading = prop_df.rename(columns = {'Training4': 'Training4_Fading', 'Reversal1': 'Reversal1_Fading'})
## Insert proportion of fading ensembles on Training4 and Reversal1 into df
cr_df.insert(10, 'Training4_Fading', prop_df_fading['Training4_Fading'])
cr_df.insert(11, 'Reversal1_Fading', prop_df_fading['Reversal1_Fading'])

In [ ]:
## Linear regression
lm_cr_prop = pg.linear_regression(cr_df['Reversal1_Fading'], cr_df['Reversal1'], as_dataframe = False)
## Make plot
fig = pf.custom_graph_template(title = '', x_title = 'Proportion Fading Ensembles', y_title = 'Peak Correct Rejection Rate')
fig.add_trace(go.Scatter(x = cr_df['Reversal1_Fading'], y = cr_df['Reversal1'], mode = 'markers', 
                         marker_color = 'red', marker = {'line': {'color': 'black', 'width' : 0.25}}, showlegend = False))
fig.add_trace(go.Scatter(x = cr_df['Reversal1_Fading'], y = lm_cr_prop['pred'], mode = 'lines', line_color = 'black', showlegend = False))
fig.show()
fig.write_image(pjoin(figure_path, 'cr_vs_prop_fading_peak.png'))

Bin activations by FORWARD trials using old behavior data input.

In [50]:
## Set mouse list, across_time, x_bin_size
mouse_list = ['mc03', 'mc06', 'mc07', 'mc09', 'mc11'] 
across_time = False
x_bin_size = 15

## Create empty dictionaries
mouse_trends = {}
mouse_binned_activations = {}
mouse_slopes = {}
mouse_taus = {}
## Loop through each mouse
for mouse in tqdm(mouse_list):
    ## saved ensemble data path
    spath = pjoin(ensemble_path, mouse)
    ## load in behavior data across all sessions
    if not across_time:
        mouse_behavior = ctb.import_mouse_behavior_data(behavior_path, mouse, key_file = 'behavior_keys.yml', session = '30min')
    ## Create empty dictionaries to store output
    determined_trends = {}
    binned_activations_dict = {}
    slopes_dict = {}
    tau_dict = {}
    ## Load assemblies
    for session in session_dict[mouse]:
        ## Load specific session's assemblies
        assemblies = ica.load_session_assemblies(mouse, spath = spath, session_id = session)
        act = assemblies['activations'].values
        ## Load a specific session's behavior data
        if not across_time:
            aligned_behavior = mouse_behavior[session]
            ## Ensure that activations and aligned behavior are the same length
            aligned_behavior = ica.align_activations_to_behavior(act, aligned_behavior)
            ## Get which timestamps are part of which trial for the aligned behavior data
            trials = ctb.get_trials(aligned_behavior, shift_factor = 0, angle_type = 'radians', counterclockwise = True)
        if across_time:
          trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_time(act, z_threshold = None, x_bin_size = x_bin_size, analysis_type = 'max', zscored = True, alpha = 'sidak')  
        else:
            ## Define ensemble trends across trials to determine if activation strength is increasing/decreasing across the session
            trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_trials(act, aligned_behavior, trials, trial_type = 'forward', z_threshold = None, zscored = False)
        ## Save to dictionaries
        determined_trends[session] = trends
        binned_activations_dict[session] = binned_activations
        slopes_dict[session] = slopes
        tau_dict[session] = tau
    ## Determine the proportion of ensembles that are increasing, decreasing, or have no trend based on their activation strength across time
    proportion_dict = ica.calculate_proportions_ensembles(determined_trends)
    ## Save to mouse dictionaries before looping to the next mouse
    mouse_trends[mouse] = proportion_dict
    mouse_binned_activations[mouse] = binned_activations_dict
    mouse_slopes[mouse] = slopes_dict
    mouse_taus[mouse] = tau_dict

  0%|          | 0/5 [00:00<?, ?it/s]/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/EnsembleRemodeling_Resubmission/../circletrack_behavior.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.timestamp = (pd.to_numeric(data.timestamp) - start_time)
/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/EnsembleRemodeling_Resubmission/../circletrack_behavior.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.timestamp = (pd.to_numeric(data.timestamp) - start_tim

In [57]:
## Get proportion decreasing values
prop_decreasing_T4 = []
prop_decreasing_R1 = []
mouse_name = []
for key in mouse_trends:
    prop_decreasing_T4.append(mouse_trends[key]['Training4']['prop_decreasing'])
    prop_decreasing_R1.append(mouse_trends[key]['Reversal1']['prop_decreasing'])
    mouse_name.append(key)
prop_df = pd.DataFrame({'Mouse': mouse_name, 'Training4': prop_decreasing_T4, 'Reversal1': prop_decreasing_R1})
## Plot prop_df
x = ['Training4', 'Reversal1']
avgs = [prop_df.Training4.mean(), prop_df.Reversal1.mean()]
sem = [prop_df.Training4.sem(), prop_df.Reversal1.sem()]
fig = go.Figure(data = go.Bar(x = x, y = avgs, error_y = dict(type = 'data', array = sem), showlegend = False))
for row in prop_df.index:
    data = prop_df.loc[row]
    fig.add_trace(go.Scatter(x = x, y = [data.Training4, data.Reversal1], mode = 'lines+markers', line_color = 'grey', showlegend = False,
                             line = dict(width = 1), name = prop_df.loc[row, 'Mouse'], marker_color = 'grey'))
fig.update_layout(template = 'simple_white', width = 500, height = 500)
fig.update_layout(
    title={
        'text': 'Fading Ensembles by Forward Trials',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.update_yaxes(title = 'Proportion Fading Ensembles')
fig.update_traces(marker_color = 'red', marker_line_color = 'black', marker_line_width = 2)
fig.show()

In [66]:
mouse_trends

{'mc03': {'Training1': {'prop_increasing': 0.0, 'prop_decreasing': 0.0},
  'Training2': {'prop_increasing': 0.0,
   'prop_decreasing': 0.05263157894736842},
  'Training3': {'prop_increasing': 0.0,
   'prop_decreasing': 0.14705882352941177},
  'Training4': {'prop_increasing': 0.07142857142857142,
   'prop_decreasing': 0.023809523809523808},
  'Reversal1': {'prop_increasing': 0.0, 'prop_decreasing': 0.0},
  'Reversal4': {'prop_increasing': 0.0, 'prop_decreasing': 0.0},
  'Training_Reversal': {'prop_increasing': 0.05263157894736842,
   'prop_decreasing': 0.0}},
 'mc06': {'Training1': {'prop_increasing': 0.0, 'prop_decreasing': 0.0},
  'Training2': {'prop_increasing': 0.0, 'prop_decreasing': 0.0},
  'Training3': {'prop_increasing': 0.024390243902439025,
   'prop_decreasing': 0.0},
  'Training4': {'prop_increasing': 0.0,
   'prop_decreasing': 0.014925373134328358},
  'Reversal1': {'prop_increasing': 0.018518518518518517,
   'prop_decreasing': 0.037037037037037035},
  'Reversal2': {'prop_inc

In [65]:
## Plot peak activation of a fading ensemble
activations = mouse_binned_activations['mc09']['Reversal1']
ensemble_number = 1
fig = pf.plot_activation_strength(activations, ensemble_number = ensemble_number, figure_path = figure_path, x_bin_size = None,
                                  title = 'Fading Ensemble', y_title = 'Max Z Score', x_title = 'Trials', 
                                  file_name = 'max_strength_fading_forward_trials.png'.format(x_bin_size), marker_color = 'red')

The code block below was used to detect PCA/ICA ensembles for each mouse and subsequently save those detected ensembles.

In [ ]:
## Save assemblies of all sessions in the keys.yml file
mouse_list =  ['mc06']    
dpath = '/media/caishuman/csstorage3/Austin/CircleTrack/EnsembleRemodeling_Resubmission/circletrack_data'
spath = '/media/caishuman/csstorage3/Austin/CircleTrack/EnsembleRemodeling_Resubmission/circletrack_data/assemblies'
for mouse in mouse_list:
    mpath = pjoin(spath, mouse)
    neural_dictionary = ctn.import_mouse_neural_data(dpath, mouse, key_file = 'minian_keys.yml', session = '30min', plot_frame_usage = False)
    ica.save_detected_ensembles(mpath, mouse, neural_dictionary, binarize = True, smooth_factor = 5, nullhyp = 'circ', n_shuffles = 500)